In [ ]:
import pandas as pd

In [ ]:
outfit_df = pd.read_csv('../../data/train-test/season_new_meta_22-23.csv')

In [ ]:
# Convert the outfit_id column into a list of tuples (outfit_id, reporter)
outfit_id_list = [(outfit_id) for outfit_id in outfit_df['outfit_id']]
outfit_reporter_list = [(reporter) for reporter in outfit_df['reporter']]
print(outfit_id_list)
print(outfit_reporter_list)


In [ ]:
!pip install --upgrade jpype1

In [ ]:
import jpype
print(jpype.isJVMStarted()) #return False:not running or 0:running

In [ ]:
!apt-get update

!apt-get install g++ openjdk-8-jdk python-dev python3-dev -y

!pip3 install JPype1-py3

!pip3 install konlpy

!JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
from konlpy.tag import Kkma
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# 꼬꼬마 형태소 분석기 초기화
kkma = Kkma()

# 문장을 형태소 단위로 분리하는 함수
def tokenize(text):
    return ' '.join(kkma.morphs(text))

# 문장을 형태소 단위로 분리하여 벡터화
vectorizer = TfidfVectorizer(tokenizer=tokenize)
tfidf_matrix = vectorizer.fit_transform(outfit_reporter_list)
print(tfidf_matrix)
# 문장들 간의 유사도 계산
similarities = cosine_similarity(tfidf_matrix)

# 결과 출력 with tqdm
num_sentences = len(outfit_reporter_list)
with tqdm(total=num_sentences*(num_sentences-1)//2) as pbar:
    for i in range(len(outfit_reporter_list)):
        for j in range(i + 1, len(outfit_reporter_list)):
            similarity = similarities[i][j]
            # print(f"문장 {i+1}과 문장 {j+1}의 유사도: {similarity}")
            pbar.update(1)
            



In [ ]:
import numpy as np

# ... Your code to calculate the similarities ...

# Save the similarities to a file
np.save('similarities.npy', similarities)


In [ ]:
# Load the similarities from the file
similarities = np.load('similarities.npy')

# Now you can use the loaded similarities for further processing or analysis


In [ ]:
similarities[1][:]

In [ ]:
import numpy as np
import csv

# ... (previous code)

# Initialize a list to store the data
data = []

# Iterate over all items
for sentence_idx in range(len(outfit_reporter_list)):
    # Get the similarities for the specified item
    similarities_for_sentence = similarities[sentence_idx]

    # Sort the similarities in descending order and get the indices of the sorted similarities
    sorted_indices = np.argsort(similarities_for_sentence)[::-1]

    # Get the top 10 most similar item indices (excluding the original item)
    top_10_indices = sorted_indices[1:11]

    # Get the outfit_id for the current item
    outfit_id = outfit_id_list[sentence_idx]

    # Get the outfit_id for the top 10 most similar items
    similar_outfits = [outfit_id_list[idx] for idx in top_10_indices]

    # Append the data to the list
    data.append([outfit_id, "{" + ','.join(map(str, similar_outfits)) + "}"])

# Save the data to a CSV file
csv_filename = 'similarwithkkma.csv'
with open(csv_filename, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['outfit_id', 'similar_outfits'])
    csv_writer.writerows(data)

print(f"Data saved to {csv_filename}.")


In [ ]:
import webbrowser

outfit_ids = [86875,87482,87956,88613,87879,77980,87838,87101,88085,87060,86828]


base_url = 'https://stylesjourney.com/detail/'

for outfit_id in outfit_ids:
    url = base_url + str(outfit_id)
    webbrowser.open(url)
